### **Modelo NCF (Neural Collaborative Filtering)**

### **Pasos de Implementación**
1. **Preprocesamiento de Datos**
   - Submuestreo de interacciones donde `add_to_cart = 0`.
   - Transformación logarítmica para variables altamente desbalanceadas como `popularity` y número de interacciones por usuario/producto.

2. **Estrategias para Datos Desbalanceados**
   - Incorporar pesos en la función de pérdida para dar mayor importancia a `add_to_cart = 1`.
   - Utilizar técnicas de oversampling para la clase minoritaria.

3. **Adaptación del Modelo**
   - Crear un modelo híbrido que:
     - Use embeddings de productos (`similar_products`, `embedding`) para manejar productos nuevos o poco populares.
     - Agregue datos demográficos (como `country`) como entradas adicionales.
     - Incluya una capa de clasificación para predecir la probabilidad de `add_to_cart`.

4. **Pipeline para Usuarios Nuevos**
   - Usar un enfoque basado en contenido para usuarios sin historial (`user_id = -1`).
   - Crear recomendaciones basadas en popularidad y similitud de producto.

5. **Evaluación y Métricas**
   - Asegurar que las métricas utilizadas reflejen el desbalance de clases, como:
     - NDCG.
     - F1-Score ajustado para la clase `add_to_cart = 1`.



In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Dropout
from tensorflow.keras.regularizers import l2

# --- Cargar datos ---
train_df = pd.read_pickle("/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/train_data_enriched.pkl")
test_df = pd.read_pickle("/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/test_data.pkl")

# --- Preprocesamiento ---
# 1. Submuestreo
add_to_cart_0 = train_df[train_df['add_to_cart'] == 0]
add_to_cart_1 = train_df[train_df['add_to_cart'] == 1]
subsampled_0 = add_to_cart_0.sample(n=len(add_to_cart_1) * 2, random_state=42)
train_balanced = pd.concat([subsampled_0, add_to_cart_1]).sample(frac=1, random_state=42)

# 2. Transformación logarítmica y normalización
for col in ['popularity', 'session_interactions']:
    train_balanced[col] = np.log1p(train_balanced[col])
    train_balanced[col] = (train_balanced[col] - train_balanced[col].mean()) / train_balanced[col].std()

# 3. Codificación de datos categóricos
train_balanced['country'] = train_balanced['country'].astype('category').cat.codes
train_balanced['pagetype'] = train_balanced['pagetype'].astype('category').cat.codes

# 4. Reasignar índices
user_mapping = {id_: idx for idx, id_ in enumerate(train_balanced['user_id'].unique())}
item_mapping = {id_: idx for idx, id_ in enumerate(train_balanced['partnumber'].unique())}
train_balanced['user_id'] = train_balanced['user_id'].map(user_mapping).fillna(len(user_mapping)).astype(int)
train_balanced['partnumber'] = train_balanced['partnumber'].map(item_mapping).fillna(len(item_mapping)).astype(int)

# --- Modelo ---
num_users = train_balanced['user_id'].nunique() + 1
num_items = train_balanced['partnumber'].nunique() + 1
num_countries = train_balanced['country'].nunique() + 1
num_pagetype = train_balanced['pagetype'].nunique() + 1

# Entradas
user_input = Input(shape=(1,), name='user_input')
item_input = Input(shape=(1,), name='item_input')
country_input = Input(shape=(1,), name='country_input')
pagetype_input = Input(shape=(1,), name='pagetype_input')

# Embeddings con regularización L2
user_embedding = Embedding(num_users, 64, name='user_embedding', embeddings_regularizer=l2(1e-4))(user_input)
item_embedding = Embedding(num_items, 64, name='item_embedding', embeddings_regularizer=l2(1e-4))(item_input)
country_embedding = Embedding(num_countries, 16, name='country_embedding', embeddings_regularizer=l2(1e-4))(country_input)
pagetype_embedding = Embedding(num_pagetype, 16, name='pagetype_embedding', embeddings_regularizer=l2(1e-4))(pagetype_input)

# Flatten layers
user_flatten = Flatten()(user_embedding)
item_flatten = Flatten()(item_embedding)
country_flatten = Flatten()(country_embedding)
pagetype_flatten = Flatten()(pagetype_embedding)

# Concatenación
concat = Concatenate()([user_flatten, item_flatten, country_flatten, pagetype_flatten])

# Capas densas con Dropout y regularización L2
fc1 = Dense(128, activation='relu', kernel_regularizer=l2(1e-4))(concat)
fc1 = Dropout(0.4)(fc1)
fc2 = Dense(64, activation='relu', kernel_regularizer=l2(1e-4))(fc1)
fc2 = Dropout(0.4)(fc2)
output = Dense(1, activation='sigmoid', name='output')(fc2)

model = Model(inputs=[user_input, item_input, country_input, pagetype_input], outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# --- Entrenamiento ---
X = train_balanced[['user_id', 'partnumber', 'country', 'pagetype']]
y = train_balanced['add_to_cart']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    [
        X_train['user_id'].values,
        X_train['partnumber'].values,
        X_train['country'].values,
        X_train['pagetype'].values
    ],
    y_train.values,
    validation_data=(
        [
            X_val['user_id'].values,
            X_val['partnumber'].values,
            X_val['country'].values,
            X_val['pagetype'].values
        ],
        y_val.values
    ),
    epochs=10,
    batch_size=512,
    class_weight=class_weight_dict,
    callbacks=[early_stopping]
)

# --- Evaluación ---
eval_results = model.evaluate(
    [
        X_val['user_id'].values,
        X_val['partnumber'].values,
        X_val['country'].values,
        X_val['pagetype'].values
    ],
    y_val.values
)
print("\nResultados de evaluación:", eval_results)


In [7]:
# Guardar el modelo en el formato nativo de Keras (recomendado)
model.save("/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/models/recommender_model_v2.keras")


---

### Ajustes Clave

1. **Usar Embeddings**:
   - Incorporar los `reduced_embedding` directamente en el modelo como entrada para capturar relaciones entre productos.

2. **Incluir Variables Categóricas**:
   - Añadir `color_id`, `family`, y `cod_section` como entradas adicionales.

3. **Incluir Información de Popularidad y Descuento**:
   - Usar `popularity` y `discount` como entradas normalizadas.

4. **Procesar Similaridades entre Productos**:
   - Usar `similar_products` para diversificar recomendaciones.


### Beneficios del Ajuste

1. **Embeddings de Producto**:
   - Captura relaciones entre productos utilizando `reduced_embedding`.

2. **Variables Adicionales**:
   - `color_id`, `family`, y `cod_section` agregan contexto para recomendaciones.

3. **Personalización Mejorada**:
   - Variables como `popularity` y `discount` ayudan a ajustar recomendaciones basadas en características importantes.

In [ ]:
import tensorflow as tf
print("TensorFlow CUDA Version:", tf.sysconfig.get_build_info()['cuda_version'])
print("TensorFlow cuDNN Version:", tf.sysconfig.get_build_info()['cudnn_version'])


In [ ]:
import tensorflow as tf

print("TensorFlow versión:", tf.__version__)
print("GPU detectada:", tf.config.list_physical_devices('GPU'))

# Prueba con una operación básica
with tf.device('/GPU:0'):
    a = tf.random.normal([1000, 1000])
    b = tf.random.normal([1000, 1000])
    c = tf.matmul(a, b)
print("Operación de prueba realizada en GPU con éxito.")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Dropout

# --- Cargar el DataFrame enriquecido ---
train_df = pd.read_pickle('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/train_data_final.pkl')

# --- Verificación y mapeo de índices ---
user_mapping = {id_: idx for idx, id_ in enumerate(train_df['user_id'].unique())}
item_mapping = {id_: idx for idx, id_ in enumerate(train_df['partnumber'].unique())}
color_mapping = {id_: idx for idx, id_ in enumerate(train_df['color_id'].unique())}
family_mapping = {id_: idx for idx, id_ in enumerate(train_df['family'].unique())}
cod_section_mapping = {id_: idx for idx, id_ in enumerate(train_df['cod_section'].unique())}
cluster_mapping = {id_: idx for idx, id_ in enumerate(train_df['cluster'].unique())}

# Aplicar mapeo
train_df['user_id'] = train_df['user_id'].map(user_mapping).fillna(0).astype(int)
train_df['partnumber'] = train_df['partnumber'].map(item_mapping).fillna(0).astype(int)
train_df['color_id'] = train_df['color_id'].map(color_mapping).fillna(0).astype(int)
train_df['family'] = train_df['family'].map(family_mapping).fillna(0).astype(int)
train_df['cod_section'] = train_df['cod_section'].map(cod_section_mapping).fillna(0).astype(int)
train_df['cluster'] = train_df['cluster'].map(cluster_mapping).fillna(0).astype(int)

# --- Verificar rangos y valores ---
print("Verificando valores fuera de rango:")
print(f"user_id max: {train_df['user_id'].max()} / {len(user_mapping)}")
print(f"partnumber max: {train_df['partnumber'].max()} / {len(item_mapping)}")
print(f"color_id max: {train_df['color_id'].max()} / {len(color_mapping)}")
print(f"family max: {train_df['family'].max()} / {len(family_mapping)}")
print(f"cod_section max: {train_df['cod_section'].max()} / {len(cod_section_mapping)}")
print(f"cluster max: {train_df['cluster'].max()} / {len(cluster_mapping)}")

# --- Preparación de los datos ---
X = train_df[['user_id', 'partnumber', 'color_id', 'family', 'cod_section', 'discount', 'popularity', 'session_interactions', 'cluster']]
y = train_df['add_to_cart']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Construcción del modelo ---
num_users = len(user_mapping) + 1
num_items = len(item_mapping) + 1
num_colors = len(color_mapping) + 1
num_families = len(family_mapping) + 1
num_cod_sections = len(cod_section_mapping) + 1
num_clusters = len(cluster_mapping) + 1

# Entradas
user_input = Input(shape=(1,), name='user_input')
item_input = Input(shape=(1,), name='item_input')
color_input = Input(shape=(1,), name='color_input')
family_input = Input(shape=(1,), name='family_input')
cod_section_input = Input(shape=(1,), name='cod_section_input')
discount_input = Input(shape=(1,), name='discount_input')
popularity_input = Input(shape=(1,), name='popularity_input')
session_input = Input(shape=(1,), name='session_input')
cluster_input = Input(shape=(1,), name='cluster_input')

# Embeddings
user_embedding = Embedding(num_users, 64)(user_input)
item_embedding = Embedding(num_items, 64)(item_input)
color_embedding = Embedding(num_colors, 16)(color_input)
family_embedding = Embedding(num_families, 16)(family_input)
cod_section_embedding = Embedding(num_cod_sections, 16)(cod_section_input)
cluster_embedding = Embedding(num_clusters, 16)(cluster_input)

# Flatten layers
user_flatten = Flatten()(user_embedding)
item_flatten = Flatten()(item_embedding)
color_flatten = Flatten()(color_embedding)
family_flatten = Flatten()(family_embedding)
cod_section_flatten = Flatten()(cod_section_embedding)
cluster_flatten = Flatten()(cluster_embedding)

# Concatenation
concat = Concatenate()([user_flatten, item_flatten, color_flatten, family_flatten, cod_section_flatten, discount_input, popularity_input, session_input, cluster_flatten])

# Fully connected layers
fc1 = Dense(128, activation='relu')(concat)
fc1 = Dropout(0.4)(fc1)
fc2 = Dense(64, activation='relu')(fc1)
fc2 = Dropout(0.4)(fc2)
output = Dense(1, activation='sigmoid')(fc2)

model = Model(inputs=[user_input, item_input, color_input, family_input, cod_section_input, discount_input, popularity_input, session_input, cluster_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'AUC'])

# Pesos de clase
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))

# --- Entrenamiento del modelo ---
history = model.fit(
    [
        X_train['user_id'].values,
        X_train['partnumber'].values,
        X_train['color_id'].values,
        X_train['family'].values,
        X_train['cod_section'].values,
        X_train['discount'].values,
        X_train['popularity'].values,
        X_train['session_interactions'].values,
        X_train['cluster'].values
    ],
    y_train.values,
    validation_data=(
        [
            X_val['user_id'].values,
            X_val['partnumber'].values,
            X_val['color_id'].values,
            X_val['family'].values,
            X_val['cod_section'].values,
            X_val['discount'].values,
            X_val['popularity'].values,
            X_val['session_interactions'].values,
            X_val['cluster'].values
        ],
        y_val.values
    ),
    epochs=10,
    batch_size=512,
    class_weight=class_weight_dict,
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
)

# --- Evaluación del modelo ---
eval_results = model.evaluate(
    [
        X_val['user_id'].values,
        X_val['partnumber'].values,
        X_val['color_id'].values,
        X_val['family'].values,
        X_val['cod_section'].values,
        X_val['discount'].values,
        X_val['popularity'].values,
        X_val['session_interactions'].values,
        X_val['cluster'].values
    ],
    y_val.values
)
print("\nResultados de evaluación:", eval_results)


In [5]:
# Guardar el modelo en el formato nativo de Keras (recomendado)
model.save("/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/models/recommender_model_v3.keras")

---

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Dropout


# --- Cargar el DataFrame enriquecido ---
train_df = pd.read_pickle('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/train_data_final.pkl')

# --- Preparación de los datos ---
X = train_df[['user_id', 'partnumber', 'color_id', 'family', 'cod_section', 'discount', 'popularity', 'session_interactions', 'cluster']]
y = train_df['add_to_cart']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Análisis para implementar mejoras



### **1. Análisis de importancia de características**
Usaremos técnicas como **árboles de decisión** o **SHAP** para medir la contribución relativa de cada característica en el modelo. Esto nos ayudará a identificar características redundantes o poco relevantes.

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt

# Definir la proporción de datos a usar
sample_fraction = 0.1

# Seleccionar una muestra aleatoria del 10% del conjunto de entrenamiento
train_sample_idx = np.random.choice(X_train.index, size=int(len(X_train) * sample_fraction), replace=False)

# Entrenar el modelo con la muestra seleccionada
rf_model = RandomForestClassifier(n_estimators=100, n_jobs=4, random_state=42)
rf_model.fit(X_train.loc[train_sample_idx], y_train.loc[train_sample_idx])

# Importancia de características
feature_importances = rf_model.feature_importances_
sorted_idx = np.argsort(feature_importances)

# Gráfica de importancia de características
plt.figure(figsize=(10, 6))
plt.barh(X_train.columns[sorted_idx], feature_importances[sorted_idx])
plt.xlabel('Importancia')
plt.title('Importancia de Características (Random Forest)')
plt.show()

# Seleccionar una muestra aleatoria del 10% del conjunto de validación
val_sample_idx = np.random.choice(X_val.index, size=int(len(X_val) * sample_fraction), replace=False)

# Análisis de importancia por permutación
perm_importance = permutation_importance(rf_model, X_val.loc[val_sample_idx], y_val.loc[val_sample_idx], random_state=42)
sorted_perm_idx = perm_importance.importances_mean.argsort()

plt.figure(figsize=(10, 6))
plt.barh(X_train.columns[sorted_perm_idx], perm_importance.importances_mean[sorted_perm_idx])
plt.xlabel('Importancia')
plt.title('Importancia de Características (Permutación)')
plt.show()


### **2. Análisis del impacto del balanceo de clases**
Comprobemos cómo el desbalance afecta el aprendizaje del modelo y evaluemos el impacto del balanceo con **submuestreo** o **SMOTE**.

Submuestreo de la clase mayoritaria:

In [ ]:
from sklearn.utils import resample

# Separar clases
majority = train_df[train_df['add_to_cart'] == 0]
minority = train_df[train_df['add_to_cart'] == 1]

# Submuestreo
majority_downsampled = resample(majority, replace=False, n_samples=len(minority), random_state=42)

# Combinar datos balanceados
balanced_train_df = pd.concat([majority_downsampled, minority])

# Revisar proporciones
print("Proporciones tras submuestreo:")
print(balanced_train_df['add_to_cart'].value_counts())


Aplicación de SMOTE:

In [ ]:
from imblearn.over_sampling import SMOTE

# Balancear con SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Confirmar nuevas proporciones
print("Proporciones tras SMOTE:")
print(np.unique(y_resampled, return_counts=True))


### **3. Curvas de ROC y Precision-Recall**
Las curvas ROC y Precision-Recall son útiles para evaluar el rendimiento general y ajustar el umbral de clasificación.

In [ ]:
from tensorflow.keras.models import load_model

# Ruta al archivo del modelo guardado
model_path = "/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/models/recommender_model_v3.keras"


# Cargar el modelo
model = load_model(model_path)

In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc

# Predicciones en el conjunto de validación
y_pred_proba = model.predict([
    X_val['user_id'].values,
    X_val['partnumber'].values,
    X_val['color_id'].values,
    X_val['family'].values,
    X_val['cod_section'].values,
    X_val['discount'].values,
    X_val['popularity'].values,
    X_val['session_interactions'].values,
    X_val['cluster'].values
])

# ROC
fpr, tpr, _ = roc_curve(y_val, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend()
plt.show()

# Precision-Recall
precision, recall, _ = precision_recall_curve(y_val, y_pred_proba)
pr_auc = auc(recall, precision)

plt.figure(figsize=(10, 6))
plt.plot(recall, precision, label=f'Precision-Recall curve (AUC = {pr_auc:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva Precision-Recall')
plt.legend()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend()
plt.show()

# Precision-Recall
precision, recall, _ = precision_recall_curve(y_val, y_pred_proba)
pr_auc = auc(recall, precision)

plt.figure(figsize=(10, 6))
plt.plot(recall, precision, label=f'Precision-Recall curve (AUC = {pr_auc:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva Precision-Recall')
plt.legend()
plt.show()

---

In [ ]:
datetime_columns = X_train.select_dtypes(include=['datetime', 'datetime64']).columns
print(f"Columnas datetime: {datetime_columns}")


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_curve, precision_recall_curve, auc
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Cargar y ajustar el dataset
train_df = pd.read_pickle('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/train_data_final.pkl')

# Reducción de características
train_df = train_df.drop(['discount', 'cod_section'], axis=1)

# Convertir columnas datetime a numéricas
train_df['date'] = pd.to_datetime(train_df['date']).astype(int) / 10**9
train_df['timestamp_local'] = pd.to_datetime(train_df['timestamp_local']).astype(int) / 10**9

# Balanceo de clases
majority = train_df[train_df['add_to_cart'] == 0]
minority = train_df[train_df['add_to_cart'] == 1]

majority_downsampled = majority.sample(n=len(minority), random_state=42)
balanced_train_df = pd.concat([majority_downsampled, minority])

# Separar características y etiquetas
X = balanced_train_df.drop('add_to_cart', axis=1)
y = balanced_train_df['add_to_cart']

# Codificar columnas categóricas y listas si existen
label_encoders = {}
for column in X.columns:
    if X[column].dtype == 'object' or isinstance(X[column].iloc[0], list):
        le = LabelEncoder()
        X[column] = X[column].astype(str).apply(lambda x: ' '.join(map(str, x)) if isinstance(x, list) else x)
        X[column] = le.fit_transform(X[column])
        label_encoders[column] = le

# Dividir datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Generar interacciones
X_train['session_popularity'] = X_train['session_interactions'] * X_train['popularity']
X_val['session_popularity'] = X_val['session_interactions'] * X_val['popularity']

X_train['user_item'] = X_train['user_id'] * X_train['partnumber']
X_val['user_item'] = X_val['user_id'] * X_val['partnumber']

# Hiperparámetros para búsqueda aleatoria
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Modelo base
rf = RandomForestClassifier(random_state=42, n_jobs=4)

# Búsqueda aleatoria
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                               n_iter=20, cv=3, verbose=2, random_state=42, n_jobs=4)

rf_random.fit(X_train, y_train)

# Mejor modelo
best_rf = rf_random.best_estimator_

# Predicciones
y_pred_proba = best_rf.predict_proba(X_val)[:, 1]
y_pred = best_rf.predict(X_val)

# Evaluación
roc_auc = roc_auc_score(y_val, y_pred_proba)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)
mcc = matthews_corrcoef(y_val, y_pred)

print("ROC AUC:", roc_auc)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Matthews Correlation Coefficient:", mcc)

# Curva ROC
fpr, tpr, _ = roc_curve(y_val, y_pred_proba)
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend()
plt.show()

# Curva Precision-Recall
precision, recall, _ = precision_recall_curve(y_val, y_pred_proba)
pr_auc = auc(recall, precision)
plt.figure(figsize=(10, 6))
plt.plot(recall, precision, label=f'Precision-Recall curve (AUC = {pr_auc:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva Precision-Recall')
plt.legend()
plt.show()

# Validación cruzada
cv_scores = cross_val_score(best_rf, X_train, y_train, cv=5, scoring='roc_auc')
print("Validación cruzada AUC promedio:", np.mean(cv_scores))


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, matthews_corrcoef, roc_curve, precision_recall_curve, auc
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Cargar y ajustar el dataset
train_df = pd.read_pickle('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/train_data_final.pkl')

# Seleccionar una muestra aleatoria del 10% del dataset para pruebas
sample_fraction = 0.1
train_df = train_df.sample(frac=sample_fraction, random_state=42)

# Reducción de características
train_df = train_df.drop(['discount', 'cod_section'], axis=1)

# Convertir columnas datetime a numéricas
train_df['date'] = pd.to_datetime(train_df['date']).astype(int) / 10**9
train_df['timestamp_local'] = pd.to_datetime(train_df['timestamp_local']).astype(int) / 10**9

# Balanceo de clases
majority = train_df[train_df['add_to_cart'] == 0]
minority = train_df[train_df['add_to_cart'] == 1]

majority_downsampled = majority.sample(n=len(minority), random_state=42)
balanced_train_df = pd.concat([majority_downsampled, minority])

# Separar características y etiquetas
X = balanced_train_df.drop('add_to_cart', axis=1)
y = balanced_train_df['add_to_cart']

# Codificar columnas categóricas y listas si existen
label_encoders = {}
for column in X.columns:
    if X[column].dtype == 'object' or isinstance(X[column].iloc[0], list):
        le = LabelEncoder()
        X[column] = X[column].astype(str).apply(lambda x: ' '.join(map(str, x)) if isinstance(x, list) else x)
        X[column] = le.fit_transform(X[column])
        label_encoders[column] = le

# Dividir datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Generar interacciones
X_train['session_popularity'] = X_train['session_interactions'] * X_train['popularity']
X_val['session_popularity'] = X_val['session_interactions'] * X_val['popularity']

X_train['user_item'] = X_train['user_id'] * X_train['partnumber']
X_val['user_item'] = X_val['user_id'] * X_val['partnumber']

# Hiperparámetros para búsqueda aleatoria
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Modelo base
rf = RandomForestClassifier(random_state=42, n_jobs=4)

# Búsqueda aleatoria
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                               n_iter=20, cv=3, verbose=2, random_state=42, n_jobs=4)

rf_random.fit(X_train, y_train)

# Mejor modelo
best_rf = rf_random.best_estimator_

# Predicciones
y_pred_proba = best_rf.predict_proba(X_val)[:, 1]
y_pred = best_rf.predict(X_val)

# Evaluación
roc_auc = roc_auc_score(y_val, y_pred_proba)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred)
mcc = matthews_corrcoef(y_val, y_pred)

print("ROC AUC:", roc_auc)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Matthews Correlation Coefficient:", mcc)

# Curva ROC
fpr, tpr, _ = roc_curve(y_val, y_pred_proba)
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Curva ROC')
plt.legend()
plt.show()

# Curva Precision-Recall
precision, recall, _ = precision_recall_curve(y_val, y_pred_proba)
pr_auc = auc(recall, precision)
plt.figure(figsize=(10, 6))
plt.plot(recall, precision, label=f'Precision-Recall curve (AUC = {pr_auc:.2f})')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Curva Precision-Recall')
plt.legend()
plt.show()

# Validación cruzada
cv_scores = cross_val_score(best_rf, X_train, y_train, cv=5, scoring='roc_auc')
print("Validación cruzada AUC promedio:", np.mean(cv_scores))


### **Evaluación de los Resultados**

#### **Métricas del Modelo**

1. **ROC AUC (Conjunto de Validación):** 0.70
   - Indica que el modelo tiene una capacidad moderada para distinguir entre positivos y negativos. En el rango de 0.5 a 1, esto es un desempeño decente para un modelo inicial, aunque hay espacio para mejora.
   
2. **Precisión:** 0.65
   - De cada predicción positiva realizada por el modelo, aproximadamente el 65% son correctas. Esto significa que hay falsos positivos que deberían ser reducidos para mejorar la precisión.

3. **Recall:** 0.62
   - El modelo identifica correctamente el 62% de los verdaderos positivos. Aunque este es un buen inicio, trabajar en mejorar el recall ayudaría a capturar más interacciones positivas (productos añadidos al carrito).

4. **F1 Score:** 0.63
   - Como métrica de balance entre precisión y recall, este valor indica un desempeño moderado. A medida que el modelo se afine, deberíamos buscar incrementarlo.

5. **MCC (Matthews Correlation Coefficient):** 0.28
   - Una métrica más estricta que combina las tasas de falsos positivos, falsos negativos, verdaderos positivos y verdaderos negativos. Aunque positiva, este valor sugiere que hay margen considerable para mejora.

6. **Validación cruzada AUC promedio:** 0.70
   - Este resultado consistente con el ROC AUC del conjunto de validación sugiere que el modelo no está sobreajustado y que los resultados son estables en diferentes particiones de los datos.

---

#### **Gráficas**

1. **Curva ROC**
   - El AUC de 0.70 refleja una separación moderada entre las clases. 
   - **Interpretación:** El modelo tiene un desempeño razonable para maximizar los verdaderos positivos mientras minimiza los falsos positivos.
   - **Mejora:** Podríamos intentar ajustes de hiperparámetros más detallados, técnicas de ensamble o agregar más features para mejorar la capacidad de discriminación.

2. **Curva Precision-Recall**
   - El AUC de 0.72 muestra que el modelo tiene un desempeño razonable en un escenario desequilibrado, donde las clases positivas (productos añadidos al carrito) podrían ser menos frecuentes.
   - **Interpretación:** La curva decrece rápidamente, lo que indica que a medida que el recall aumenta, la precisión disminuye. Esto es típico cuando el modelo está intentando maximizar el recall.
   - **Mejora:** Podríamos intentar estrategias para manejar el desequilibrio de clases (e.g., cambiar el umbral de decisión, o técnicas de re-muestreo más avanzadas).

---

#### **Análisis Final**

- El modelo tiene un **desempeño inicial prometedor**, pero hay margen para optimización:
  - Experimentar con diferentes arquitecturas de modelos, como ensambles (e.g., Gradient Boosting, XGBoost) o redes neuronales.
  - Investigar características adicionales, como embeddings más detallados o combinaciones de variables que capten mejor las interacciones usuario-producto.
  - Aplicar técnicas avanzadas de ajuste de hiperparámetros, como `GridSearchCV` o `Bayesian Optimization`.

- El **dataset reducido (10%)** permitió completar la ejecución con tiempos razonables, pero sería ideal escalar el modelo completo para obtener resultados más robustos.


---

Dado que el dataset de entrenamiento es grande y el modelo de **Random Forest** puede no ser ideal en estos escenarios, ajustar el modelo basado en **embeddings** es una decisión estratégica sólida. Aquí te detallo cómo podríamos proceder para optimizar y validar este modelo:

---

### **Ajustes y Mejoras para el Modelo de Embeddings**

1. **Incrementar el Tamaño de las Embeddings**
   - Actualmente tienes embeddings de tamaño:
     - `user_id` y `partnumber`: **64**
     - Otras variables categóricas: **16**
   - Considera aumentar las dimensiones de las embeddings para usuarios y productos a **128**, especialmente si tienes muchos usuarios/productos únicos.
   - Otras categorías podrían incrementarse a **32** dependiendo del número de valores únicos.

   ```python
   user_embedding = Embedding(num_users, 128)(user_input)
   item_embedding = Embedding(num_items, 128)(item_input)
   ```

2. **Capas Densas y Regularización**
   - Agregar una capa adicional al modelo denso puede ayudar a capturar relaciones más complejas.
   - Aumenta el `Dropout` si notas señales de sobreajuste (reduce las métricas en `val_loss` mientras `loss` mejora).

   ```python
   fc3 = Dense(32, activation='relu')(fc2)
   fc3 = Dropout(0.5)(fc3)
   output = Dense(1, activation='sigmoid')(fc3)
   ```

3. **Ajustar el Optimizador**
   - Cambia de `Adam` a otros optimizadores como:
     - **AdamW**: Incluye regularización L2 automáticamente.
     - **SGD** con `momentum`: Más estable para datasets grandes, pero más lento.

   ```python
   model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=0.001),
                 loss='binary_crossentropy', metrics=['accuracy', 'AUC'])
   ```

4. **Callback de Reducción del Learning Rate**
   - Introduce un callback para reducir automáticamente el `learning rate` si no mejora la métrica en el conjunto de validación.

   ```python
   callbacks = [
       tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
       tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
   ]
   ```

5. **Clase `BatchNormalization`**
   - Inserta una capa de normalización después de la concatenación para estabilizar el entrenamiento.

   ```python
   concat = BatchNormalization()(concat)
   ```

6. **Pesos de Clases**
   - Mantén el uso de `class_weight` para balancear el desequilibrio de clases en el dataset.

---

### **Plan de Validación**

1. **K-Fold Cross Validation**
   - Divide el dataset en **k** particiones (e.g., `k=5`) para evaluar el modelo en cada partición y obtener una métrica promedio (como AUC).

   ```python
   from sklearn.model_selection import KFold
   kf = KFold(n_splits=5, shuffle=True, random_state=42)
   ```

2. **Métricas de Evaluación**
   - Genera métricas como:
     - **AUC** y **Accuracy** en validación.
     - Curvas ROC y Precision-Recall.
   - Esto permitirá una comparación directa con modelos previos.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import AdamW

# Configurar para que el crecimiento de memoria sea dinámico
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


# --- Cargar el DataFrame enriquecido ---
train_df = pd.read_pickle('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/train_data_final.pkl')

# --- Verificación y mapeo de índices ---
user_mapping = {id_: idx for idx, id_ in enumerate(train_df['user_id'].unique())}
item_mapping = {id_: idx for idx, id_ in enumerate(train_df['partnumber'].unique())}
color_mapping = {id_: idx for idx, id_ in enumerate(train_df['color_id'].unique())}
family_mapping = {id_: idx for idx, id_ in enumerate(train_df['family'].unique())}
cod_section_mapping = {id_: idx for idx, id_ in enumerate(train_df['cod_section'].unique())}
cluster_mapping = {id_: idx for idx, id_ in enumerate(train_df['cluster'].unique())}

# Aplicar mapeo
train_df['user_id'] = train_df['user_id'].map(user_mapping).fillna(0).astype(int)
train_df['partnumber'] = train_df['partnumber'].map(item_mapping).fillna(0).astype(int)
train_df['color_id'] = train_df['color_id'].map(color_mapping).fillna(0).astype(int)
train_df['family'] = train_df['family'].map(family_mapping).fillna(0).astype(int)
train_df['cod_section'] = train_df['cod_section'].map(cod_section_mapping).fillna(0).astype(int)
train_df['cluster'] = train_df['cluster'].map(cluster_mapping).fillna(0).astype(int)

# --- Verificar rangos y valores ---
print("Verificando valores fuera de rango:")
print(f"user_id max: {train_df['user_id'].max()} / {len(user_mapping)}")
print(f"partnumber max: {train_df['partnumber'].max()} / {len(item_mapping)}")
print(f"color_id max: {train_df['color_id'].max()} / {len(color_mapping)}")
print(f"family max: {train_df['family'].max()} / {len(family_mapping)}")
print(f"cod_section max: {train_df['cod_section'].max()} / {len(cod_section_mapping)}")
print(f"cluster max: {train_df['cluster'].max()} / {len(cluster_mapping)}")

# --- Preparación de los datos ---
X = train_df[['user_id', 'partnumber', 'color_id', 'family', 'cod_section', 'discount', 'popularity', 'session_interactions', 'cluster']]
y = train_df['add_to_cart']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Dimensiones del modelo ---
num_users = len(user_mapping) + 1
num_items = len(item_mapping) + 1
num_colors = len(color_mapping) + 1
num_families = len(family_mapping) + 1
num_cod_sections = len(cod_section_mapping) + 1
num_clusters = len(cluster_mapping) + 1

# --- Construcción del modelo optimizado ---
user_input = Input(shape=(1,), name='user_input')
item_input = Input(shape=(1,), name='item_input')
color_input = Input(shape=(1,), name='color_input')
family_input = Input(shape=(1,), name='family_input')
cod_section_input = Input(shape=(1,), name='cod_section_input')
discount_input = Input(shape=(1,), name='discount_input')
popularity_input = Input(shape=(1,), name='popularity_input')
session_input = Input(shape=(1,), name='session_input')
cluster_input = Input(shape=(1,), name='cluster_input')

# Embeddings
user_embedding = Embedding(num_users, 128)(user_input)
item_embedding = Embedding(num_items, 128)(item_input)
color_embedding = Embedding(num_colors, 32)(color_input)
family_embedding = Embedding(num_families, 32)(family_input)
cod_section_embedding = Embedding(num_cod_sections, 32)(cod_section_input)
cluster_embedding = Embedding(num_clusters, 32)(cluster_input)

# Flatten layers
user_flatten = Flatten()(user_embedding)
item_flatten = Flatten()(item_embedding)
color_flatten = Flatten()(color_embedding)
family_flatten = Flatten()(family_embedding)
cod_section_flatten = Flatten()(cod_section_embedding)
cluster_flatten = Flatten()(cluster_embedding)

# Concatenation
concat = Concatenate()([user_flatten, item_flatten, color_flatten, family_flatten, cod_section_flatten,
                        discount_input, popularity_input, session_input, cluster_flatten])
concat = BatchNormalization()(concat)

# Fully connected layers
fc1 = Dense(256, activation='relu')(concat)
fc1 = Dropout(0.5)(fc1)
fc2 = Dense(128, activation='relu')(fc1)
fc2 = Dropout(0.5)(fc2)
fc3 = Dense(32, activation='relu')(fc2)
fc3 = Dropout(0.5)(fc3)
output = Dense(1, activation='sigmoid')(fc3)

model = Model(inputs=[user_input, item_input, color_input, family_input, cod_section_input,
                      discount_input, popularity_input, session_input, cluster_input], outputs=output)
model.compile(optimizer=AdamW(learning_rate=0.001),
              loss='binary_crossentropy', metrics=['accuracy', 'AUC'])

# --- Pesos de clase ---
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))

# --- Entrenamiento ---
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)
]

history = model.fit(
    [
        X_train['user_id'].values, X_train['partnumber'].values, X_train['color_id'].values,
        X_train['family'].values, X_train['cod_section'].values, X_train['discount'].values,
        X_train['popularity'].values, X_train['session_interactions'].values, X_train['cluster'].values
    ],
    y_train.values,
    validation_data=(
        [
            X_val['user_id'].values, X_val['partnumber'].values, X_val['color_id'].values,
            X_val['family'].values, X_val['cod_section'].values, X_val['discount'].values,
            X_val['popularity'].values, X_val['session_interactions'].values, X_val['cluster'].values
        ],
        y_val.values
    ),
    epochs=15,
    batch_size=512,
    class_weight=class_weight_dict,
    callbacks=callbacks
)

# --- Evaluación ---
eval_results = model.evaluate(
    [
        X_val['user_id'].values, X_val['partnumber'].values, X_val['color_id'].values,
        X_val['family'].values, X_val['cod_section'].values, X_val['discount'].values,
        X_val['popularity'].values, X_val['session_interactions'].values, X_val['cluster'].values
    ],
    y_val.values
)
print("\nResultados de evaluación:", eval_results)


In [2]:
# Guardar el modelo en el formato nativo de Keras (recomendado)
model.save("/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/models/recommender_model_v4.keras")

### Análisis de Resultados

#### 1. **Métricas finales del modelo**
- **Pérdida de validación (`val_loss`)**: 0.5834
- **AUC de validación (`val_AUC`)**: 0.6694
- **Accuracy de validación (`val_accuracy`)**: 68.84%

El modelo ha logrado un rendimiento decente en términos de AUC de validación y accuracy. Sin embargo, hay algunos puntos importantes que debemos analizar:

---

#### 2. **Evolución de las métricas durante el entrenamiento**
- **AUC en validación**:
  - Comenzó en 0.6686 y alcanzó un máximo de 0.6713, pero no mostró mejoras significativas después de la tercera epoch.
  - Las últimas epochs no aportaron mejoras consistentes en la AUC, lo que sugiere que el modelo puede haber alcanzado su límite de aprendizaje para la configuración actual.

- **Pérdida de validación (`val_loss`)**:
  - Fluctuó entre 0.5834 y 0.6412 en las últimas epochs, indicando un posible ajuste excesivo o falta de aprendizaje adicional.

- **Learning Rate Decay**:
  - La reducción de la tasa de aprendizaje (a 0.0005) no tuvo un impacto significativo en la mejora de las métricas.

---

#### 3. **Aspectos técnicos del entrenamiento**
- **Advertencias relacionadas con CUDA/cuDNN**:
  - Aunque estas advertencias no detuvieron el entrenamiento, pueden haber reducido la eficiencia del hardware.
  - Es recomendable validar la configuración de CUDA/cuDNN para maximizar el rendimiento.

- **Uso de memoria GPU**:
  - La configuración para crecimiento dinámico de memoria fue efectiva, permitiendo una ejecución estable en términos de recursos.

---

#### 4. **Interpretación general**
- **Puntos positivos**:
  - La AUC en validación es consistente (~0.67), lo que muestra un buen ajuste del modelo.
  - La accuracy (~68.84%) en validación es competitiva.

- **Puntos de mejora**:
  - El modelo parece alcanzar un punto de saturación después de pocas epochs.
  - Fluctuaciones en la pérdida de validación podrían indicar un ajuste no óptimo en las capas finales o en la arquitectura del modelo.

---

### **Recomendaciones para próximos pasos**

1. **Optimización adicional del modelo**
   - **Incrementar la regularización**: Probar un aumento en los valores de Dropout o ajustar los pesos de clase.
   - **Tuning de la arquitectura**:
     - Probar con menos unidades en las capas densas (reducir 256 → 128, etc.).
     - Cambiar las funciones de activación en las capas densas, como ReLU → LeakyReLU.
   - **Optimización del Learning Rate**:
     - Usar un planificador dinámico con más control (por ejemplo, `LearningRateScheduler`).

2. **Ajustes en el preprocesamiento de datos**
   - Revisar posibles interacciones adicionales entre las características para mejorar el modelo.

3. **Validación más robusta**
   - Realizar un análisis más exhaustivo en un conjunto de test para validar la capacidad de generalización del modelo.

4. **Alternativas al modelo actual**
   - Explorar arquitecturas más avanzadas como **DeepFM** o enfoques híbridos basados en embeddings y métodos basados en grafos.


---

---

### Reentreno del modelo.

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import LearningRateScheduler

# Configurar para que el crecimiento de memoria sea dinámico
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# --- Cargar el DataFrame enriquecido ---
train_df = pd.read_pickle('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/train_data_final.pkl')

# --- Verificación y mapeo de índices ---
user_mapping = {id_: idx for idx, id_ in enumerate(train_df['user_id'].unique())}
item_mapping = {id_: idx for idx, id_ in enumerate(train_df['partnumber'].unique())}
color_mapping = {id_: idx for idx, id_ in enumerate(train_df['color_id'].unique())}
family_mapping = {id_: idx for idx, id_ in enumerate(train_df['family'].unique())}
cod_section_mapping = {id_: idx for idx, id_ in enumerate(train_df['cod_section'].unique())}
cluster_mapping = {id_: idx for idx, id_ in enumerate(train_df['cluster'].unique())}

# Aplicar mapeo
train_df['user_id'] = train_df['user_id'].map(user_mapping).fillna(0).astype(int)
train_df['partnumber'] = train_df['partnumber'].map(item_mapping).fillna(0).astype(int)
train_df['color_id'] = train_df['color_id'].map(color_mapping).fillna(0).astype(int)
train_df['family'] = train_df['family'].map(family_mapping).fillna(0).astype(int)
train_df['cod_section'] = train_df['cod_section'].map(cod_section_mapping).fillna(0).astype(int)
train_df['cluster'] = train_df['cluster'].map(cluster_mapping).fillna(0).astype(int)

# --- Preparación de los datos ---
X = train_df[['user_id', 'partnumber', 'color_id', 'family', 'cod_section', 'discount', 'popularity', 'session_interactions', 'cluster']]
y = train_df['add_to_cart']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Dimensiones del modelo ---
num_users = len(user_mapping) + 1
num_items = len(item_mapping) + 1
num_colors = len(color_mapping) + 1
num_families = len(family_mapping) + 1
num_cod_sections = len(cod_section_mapping) + 1
num_clusters = len(cluster_mapping) + 1

# --- Construcción del modelo optimizado ---
user_input = Input(shape=(1,), name='user_input')
item_input = Input(shape=(1,), name='item_input')
color_input = Input(shape=(1,), name='color_input')
family_input = Input(shape=(1,), name='family_input')
cod_section_input = Input(shape=(1,), name='cod_section_input')
discount_input = Input(shape=(1,), name='discount_input')
popularity_input = Input(shape=(1,), name='popularity_input')
session_input = Input(shape=(1,), name='session_input')
cluster_input = Input(shape=(1,), name='cluster_input')

# Embeddings
user_embedding = Embedding(num_users, 64)(user_input)
item_embedding = Embedding(num_items, 64)(item_input)
color_embedding = Embedding(num_colors, 16)(color_input)
family_embedding = Embedding(num_families, 16)(family_input)
cod_section_embedding = Embedding(num_cod_sections, 16)(cod_section_input)
cluster_embedding = Embedding(num_clusters, 16)(cluster_input)

# Flatten layers
user_flatten = Flatten()(user_embedding)
item_flatten = Flatten()(item_embedding)
color_flatten = Flatten()(color_embedding)
family_flatten = Flatten()(family_embedding)
cod_section_flatten = Flatten()(cod_section_embedding)
cluster_flatten = Flatten()(cluster_embedding)

# Concatenation
concat = Concatenate()([user_flatten, item_flatten, color_flatten, family_flatten, cod_section_flatten,
                        discount_input, popularity_input, session_input, cluster_flatten])
concat = BatchNormalization()(concat)

# Fully connected layers with LeakyReLU
fc1 = Dense(128)(concat)
fc1 = LeakyReLU(negative_slope=0.1)(fc1)
fc1 = Dropout(0.4)(fc1)
fc2 = Dense(64)(fc1)
fc2 = LeakyReLU(negative_slope=0.1)(fc2)
fc2 = Dropout(0.4)(fc2)
output = Dense(1, activation='sigmoid')(fc2)

model = Model(inputs=[user_input, item_input, color_input, family_input, cod_section_input,
                      discount_input, popularity_input, session_input, cluster_input], outputs=output)
model.compile(optimizer=AdamW(learning_rate=0.001),
              loss='binary_crossentropy', metrics=['accuracy', 'AUC'])

# --- Pesos de clase ---
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(zip(np.unique(y_train), class_weights))

# --- Scheduler para el Learning Rate ---
def lr_schedule(epoch, lr):
    if epoch > 5:
        return lr * 0.5
    return lr

# --- Entrenamiento ---
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    LearningRateScheduler(lr_schedule)
]

history = model.fit(
    [
        X_train['user_id'].values, X_train['partnumber'].values, X_train['color_id'].values,
        X_train['family'].values, X_train['cod_section'].values, X_train['discount'].values,
        X_train['popularity'].values, X_train['session_interactions'].values, X_train['cluster'].values
    ],
    y_train.values,
    validation_data=(
        [
            X_val['user_id'].values, X_val['partnumber'].values, X_val['color_id'].values,
            X_val['family'].values, X_val['cod_section'].values, X_val['discount'].values,
            X_val['popularity'].values, X_val['session_interactions'].values, X_val['cluster'].values
        ],
        y_val.values
    ),
    epochs=15,
    batch_size=512,
    class_weight=class_weight_dict,
    callbacks=callbacks
)

# --- Evaluación ---
eval_results = model.evaluate(
    [
        X_val['user_id'].values, X_val['partnumber'].values, X_val['color_id'].values,
        X_val['family'].values, X_val['cod_section'].values, X_val['discount'].values,
        X_val['popularity'].values, X_val['session_interactions'].values, X_val['cluster'].values
    ],
    y_val.values
)
print("\nResultados de evaluación:", eval_results)



Epoch 1/15
72737/72737 ━━━━━━━━━━━━━━━━━━━━ 416s 6ms/step - AUC: 0.6460 - accuracy: 0.5924 - loss: 0.6562 - val_AUC: 0.6699 - val_accuracy: 0.6419 - val_loss: 0.6481 - learning_rate: 0.0010
Epoch 2/15
72737/72737 ━━━━━━━━━━━━━━━━━━━━ 406s 6ms/step - AUC: 0.6787 - accuracy: 0.6298 - loss: 0.6374 - val_AUC: 0.6729 - val_accuracy: 0.6235 - val_loss: 0.6419 - learning_rate: 0.0010
Epoch 3/15
72737/72737 ━━━━━━━━━━━━━━━━━━━━ 404s 6ms/step - AUC: 0.6850 - accuracy: 0.6255 - loss: 0.6333 - val_AUC: 0.6727 - val_accuracy: 0.6574 - val_loss: 0.6263 - learning_rate: 0.0010
Epoch 4/15
72737/72737 ━━━━━━━━━━━━━━━━━━━━ 404s 6ms/step - AUC: 0.6869 - accuracy: 0.6240 - loss: 0.6325 - val_AUC: 0.6719 - val_accuracy: 0.5827 - val_loss: 0.6437 - learning_rate: 0.0010
Epoch 5/15
72737/72737 ━━━━━━━━━━━━━━━━━━━━ 405s 6ms/step - AUC: 0.6885 - accuracy: 0.6243 - loss: 0.6312 - val_AUC: 0.6738 - val_accuracy: 0.6105 - val_loss: 0.6316 - learning_rate: 0.0010
Epoch 6/15
72737/72737 ━━━━━━━━━━━━━━━━━━━━ 424s 6

In [18]:
# Guardar el modelo en el formato nativo de Keras
model.save("/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/models/recommender_model_v5.keras")

: 

In [ ]:
import pandas as pd
import numpy as np
import json
from tensorflow.keras.models import load_model

# --- Cargar el conjunto de prueba ---
test_df = pd.read_pickle('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/test_data.pkl')

# --- Mapear IDs disponibles ---
test_df['user_id'] = test_df['user_id'].map(user_mapping).fillna(0).astype(int)
test_df['partnumber'] = test_df['partnumber'].map(item_mapping).fillna(0).astype(int)

# --- Añadir columnas faltantes con valores por defecto ---
default_columns = ['color_id', 'family', 'cod_section', 'discount', 'popularity', 'session_interactions', 'cluster']
for col in default_columns:
    test_df[col] = 0

# --- Obtener productos populares globales ---
global_popularity = test_df['partnumber'].value_counts().index.tolist()

# --- Obtener las session_id únicas ---
session_ids = test_df['session_id'].unique()

# --- Cargar el modelo entrenado ---
model = load_model('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/models/recommender_model_v5.keras')

# --- Generar predicciones ---
predictions = {}

for session_id in session_ids:
    # Filtrar los datos de la sesión actual
    session_data = test_df[test_df['session_id'] == session_id].copy()

    # Preparar las entradas para el modelo
    user_input = session_data['user_id'].values
    item_input = session_data['partnumber'].values
    color_input = session_data['color_id'].values
    family_input = session_data['family'].values
    cod_section_input = session_data['cod_section'].values
    discount_input = session_data['discount'].values
    popularity_input = session_data['popularity'].values
    session_input = session_data['session_interactions'].values
    cluster_input = session_data['cluster'].values

    # Predicciones del modelo
    if len(user_input) > 0:
        scores = model.predict([
            user_input, item_input, color_input, family_input,
            cod_section_input, discount_input, popularity_input,
            session_input, cluster_input
        ], verbose=0)
        session_data['score'] = scores
        top_products = (
            session_data.sort_values(by='score', ascending=False)['partnumber']
            .drop_duplicates().tolist()
        )
    else:
        top_products = []

    # Rellenar con historial personalizado
    if len(top_products) < 5:
        additional_products = (
            session_data['partnumber']
            .value_counts()
            .index.tolist()
        )
        for product in additional_products:
            if product not in top_products:
                top_products.append(product)
            if len(top_products) >= 5:
                break

    # Rellenar con productos populares globales
    if len(top_products) < 5:
        for product in global_popularity:
            if product not in top_products:
                top_products.append(product)
            if len(top_products) >= 5:
                break

    # Asegurar predicciones únicas y exactas a 5 productos
    predictions[str(session_id)] = top_products[:5]

# --- Guardar el archivo predictions_3.json ---
output = {"target": predictions}

output_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/predictions/predictions_3_v5.json'
with open(output_path, 'w') as f:
    json.dump(output, f, indent=4)

print(f"Archivo predictions_3.json generado con éxito en {output_path}.")


In [ ]:
import pandas as pd
import numpy as np
import json
from tensorflow.keras.models import load_model
from collections import defaultdict

# --- Cargar el conjunto de prueba ---
test_df = pd.read_pickle('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/test_data.pkl')

# --- Mapear IDs disponibles ---
test_df['user_id'] = test_df['user_id'].map(user_mapping).fillna(0).astype(int)
test_df['partnumber'] = test_df['partnumber'].map(item_mapping).fillna(0).astype(int)

# --- Añadir columnas faltantes con valores por defecto ---
default_columns = ['color_id', 'family', 'cod_section', 'discount', 'popularity', 'session_interactions', 'cluster']
for col in default_columns:
    test_df[col] = 0

# --- Obtener productos populares globales ---
global_popularity = test_df['partnumber'].value_counts().index.tolist()

# --- Obtener las session_id únicas ---
session_ids = test_df['session_id'].unique()

# --- Cargar el modelo entrenado ---
model = load_model('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/models/recommender_model_v5.keras')

# --- Generar predicciones ---
predictions = {}

for session_id in session_ids:
    # Filtrar los datos de la sesión actual
    session_data = test_df[test_df['session_id'] == session_id].copy()

    # Preparar las entradas para el modelo
    user_input = session_data['user_id'].values
    item_input = session_data['partnumber'].values
    color_input = session_data['color_id'].values
    family_input = session_data['family'].values
    cod_section_input = session_data['cod_section'].values
    discount_input = session_data['discount'].values
    popularity_input = session_data['popularity'].values
    session_input = session_data['session_interactions'].values
    cluster_input = session_data['cluster'].values

    # Predicciones del modelo
    top_products = []
    if len(user_input) > 0:
        scores = model.predict([
            user_input, item_input, color_input, family_input,
            cod_section_input, discount_input, popularity_input,
            session_input, cluster_input
        ], verbose=0)
        session_data['score'] = scores
        top_products = (
            session_data.sort_values(by='score', ascending=False)['partnumber']
            .drop_duplicates().tolist()
        )

    # Rellenar con historial personalizado
    if len(top_products) < 5:
        additional_products = (
            session_data['partnumber']
            .value_counts()
            .index.tolist()
        )
        for product in additional_products:
            if product not in top_products:
                top_products.append(product)
            if len(top_products) >= 5:
                break

    # Rellenar con productos populares condicionados por país
    user_country = session_data['country'].iloc[0] if not session_data.empty else None
    if len(top_products) < 5 and user_country is not None:
        country_popular = (
            test_df[test_df['country'] == user_country]['partnumber']
            .value_counts()
            .index.tolist()
        )
        for product in country_popular:
            if product not in top_products:
                top_products.append(product)
            if len(top_products) >= 5:
                break

    # Completar con productos globales
    if len(top_products) < 5:
        for product in global_popularity:
            if product not in top_products:
                top_products.append(product)
            if len(top_products) >= 5:
                break

    # Asegurar predicciones únicas y exactas a 5 productos
    predictions[str(session_id)] = top_products[:5]

# --- Guardar el archivo predictions_3.json ---
output = {"target": predictions}

output_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/predictions/predictions_3_optimized_v5.json'
with open(output_path, 'w') as f:
    json.dump(output, f, indent=4)

print(f"Archivo predictions_3.json generado con éxito en {output_path}.")

---